In [50]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [51]:
#Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [58]:
batch_size = 64

train_transform = transforms.Compose(
    [transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),])

test_transform = transforms.Compose(
    [transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),])

train_set = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = train_transform)
train_loader = DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_set = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = test_transform )
test_loader = DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [59]:
# Basic convolution 
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [60]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [61]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.2)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.dropout(out)
        out = self.layer2(out)
        out = self.dropout(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [62]:
num_epochs = 80
learning_rate = 0.001

In [63]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate



for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 5
        update_lr(optimizer, curr_lr)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Epoch [1/80], Step [100/782] Loss: 0.0373
Epoch [1/80], Step [200/782] Loss: 0.0199
Epoch [1/80], Step [300/782] Loss: 0.0399
Epoch [1/80], Step [400/782] Loss: 0.0077
Epoch [1/80], Step [500/782] Loss: 0.0669
Epoch [1/80], Step [600/782] Loss: 0.0683
Epoch [1/80], Step [700/782] Loss: 0.0404
Epoch [2/80], Step [100/782] Loss: 0.0147
Epoch [2/80], Step [200/782] Loss: 0.0274
Epoch [2/80], Step [300/782] Loss: 0.0314
Epoch [2/80], Step [400/782] Loss: 0.0192
Epoch [2/80], Step [500/782] Loss: 0.0215
Epoch [2/80], Step [600/782] Loss: 0.0198
Epoch [2/80], Step [700/782] Loss: 0.0215
Epoch [3/80], Step [100/782] Loss: 0.0076
Epoch [3/80], Step [200/782] Loss: 0.0213
Epoch [3/80], Step [300/782] Loss: 0.0624
Epoch [3/80], Step [400/782] Loss: 0.0069
Epoch [3/80], Step [500/782] Loss: 0.0502
Epoch [3/80], Step [600/782] Loss: 0.0229
Epoch [3/80], Step [700/782] Loss: 0.0296
Epoch [4/80], Step [100/782] Loss: 0.0204
Epoch [4/80], Step [200/782] Loss: 0.0101
Epoch [4/80], Step [300/782] Loss: